In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time

%matplotlib widget

In [2]:
from bluesky_tutorial_utils import setup_data_saving
from bluesky import RunEngine
from bluesky.plans import count
from utils import simple_integration

RE = RunEngine()
catalog = setup_data_saving(RE)

In [3]:
from simulated_hardware import detector, load_sample, unload_sample, light, history_reset, sim_sleep


In [4]:
def dark_light_subtract(sample_num):
    #close shutter if not already closed
    yield from light(False)
        
    #move to desired sample
    yield from load_sample(sample_num)
    
    #take dark image
    yield from count([detector])
    
    #open shutter
    yield from light(True)
    
    #take light image
    yield from count([detector])
    
    #close shutter to be nice to detector
    yield from light(False)
    

def process_data(pair):
    #assuming pair is tuple
    my_dark = catalog[pair[0]].primary.read().detector_image[0]
    my_light = catalog[pair[1]].primary.read().detector_image[0]
    
    this_int = simple_integration(my_light - my_dark,num_bins=301)
    
    return this_int

### Multiple Samples and Introductory Time Travel

1.) Show how we change between samples.

2.) Measure a bunch of things in a row, then look at the last one.

3.) Talk about how we have an issue if our dark image doesn't match our current background of light (detectors have a bit of a history)

4.) Talk about need to taylor our data collection plan (frequency of darks) as a function of how strong scatterers are.  (note - this is why we avoid measuring weak scattering samples directly after strong).

5.) Make a plan based around these timing considerations, collect data, show things worked well.

In [5]:
history_reset()

RE(light(True))

RE(load_sample(1))

RE(load_sample(2))

RE(load_sample(3))



()

In [6]:
#close shutter
RE(light(False))

#measure dark
uid_dark = RE(count([detector]))

RE(load_sample(1))

RE(light(True))

#gather image sample
uid_light = RE(count([detector]))



In [7]:
my_light = catalog[uid_light].primary.read().detector_image[0]
my_dark = catalog[uid_dark].primary.read().detector_image[0]

In [8]:
plt.figure()
plt.imshow(my_light-my_dark)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
int1 = simple_integration(my_light-my_dark, num_bins = 301)
int_dark = simple_integration(my_dark, num_bins = 301)
int_light = simple_integration(my_light, num_bins = 301)

plt.figure()
plt.plot(int1)
plt.plot(int_dark)
plt.plot(int_light)
plt.plot(int_light-int_dark)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Dark has issues!  Need to be careful to let image on the detectors 'decay' between shots.

In [10]:
history_reset()

RE(light(True))

RE(load_sample(1))

RE(load_sample(2))

RE(load_sample(3))



()

In [11]:
#close shutter
RE(light(False))

sim_sleep(30)
#measure dark
uid_dark = RE(count([detector]))

RE(load_sample(1))

RE(light(True))

#gather image sample
uid_light = RE(count([detector]))



sleeping for 30


In [12]:
my_light_fixed = catalog[uid_light].primary.read().detector_image[0]
my_dark_fixed = catalog[uid_dark].primary.read().detector_image[0]

plt.figure()
plt.imshow(my_light_fixed-my_dark_fixed)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
int1 = simple_integration(my_light_fixed-my_dark_fixed, num_bins = 301)
int_dark = simple_integration(my_dark_fixed, num_bins = 301)
int_light = simple_integration(my_light_fixed, num_bins = 301)

plt.figure()
plt.plot(int1)
plt.plot(int_dark)
plt.plot(int_light)
plt.plot(int_light-int_dark)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clearly, managment of darks is an important aspect of diffraction measurements (at least on these detectors).

### Concluding remarks